### Étapes du nettoyage et du formatage des données ###

1. Chargement des données
2. Inspection des données (dimensions, types, valeurs manquantes, doublons, valeurs aberrantes)
3. Traitement des valeurs manquantes et incohérences
4. Normalisation des formats (dates, nombres, catégories)
5. Standardisation des noms de colonnes
6. Exportation des données nettoyées

In [1]:
import pandas as pd

# Charger le fichier
file_path = "C:/Users/teo11/PROJETS/walmart-sales-analysis/data/Walmart_Sales.csv"
df = pd.read_csv(file_path)

# Afficher les premières lignes pour inspection
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


### Inspection initiale des données ###
Le fichier contient les colonnes suivantes :

- Store : Identifiant du magasin
- Date : Date de la vente (format apparemment MM-JJ-AAAA)
- Weekly_Sales : Ventes hebdomadaires
- Holiday_Flag : Indicateur de jour férié (0 = non, 1 = oui)
- Temperature : Température en degrés Fahrenheit
- Fuel_Price : Prix du carburant
- CPI : Indice des prix à la consommation
- Unemployment : Taux de chômage


### Problèmes détectés ###

1. Format de la colonne "Date" : Probablement une chaîne de caractères, il faut la convertir en format datetime.
2. Vérification des valeurs manquantes et des doublons
3. Uniformisation des noms de colonnes (passage en minuscules et suppression des espaces)

Je vais effectuer ces corrections

In [2]:
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()

# Vérification des doublons
duplicate_rows = df.duplicated().sum()

# Conversion de la colonne Date en format datetime
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

# Standardisation des noms de colonnes
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Résumé des corrections effectuées
missing_values, duplicate_rows, df.dtypes

(Store           0
 Date            0
 Weekly_Sales    0
 Holiday_Flag    0
 Temperature     0
 Fuel_Price      0
 CPI             0
 Unemployment    0
 dtype: int64,
 np.int64(0),
 store                    int64
 date            datetime64[ns]
 weekly_sales           float64
 holiday_flag             int64
 temperature            float64
 fuel_price             float64
 cpi                    float64
 unemployment           float64
 dtype: object)

### Résumé du nettoyage des données ###

- Aucune valeur manquante : Toutes les colonnes sont complètes.
- Aucun doublon : Aucune ligne en double.
- Format de la colonne "Date" corrigé : Maintenant au format datetime.
- Noms de colonnes standardisés : Passage en minuscules et remplacement des espaces par des underscores.

### Identifier les premières tendances ###

Je vais réaliser les analyses suivantes avec des graphiques interactifs :

1. Évolution des ventes au fil du temps
2. Comparaison des ventes entre les magasins
3. Impact des jours fériés sur les ventes
4. Influence de la température et du prix du carburant sur les ventes.

Je vais utiliser Plotly pour créer des graphiques interactifs. Commençons par charger et préparer les données.

In [3]:
import plotly.express as px

# Recharger le fichier nettoyé pour garantir que nous utilisons les bonnes données
df_cleaned = pd.read_csv("C:/Users/teo11/PROJETS/walmart-sales-analysis/data/Walmart_Sales_Cleaned.csv")

# Vérifier que la colonne date est bien en datetime
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"])

# Afficher les premières lignes pour confirmer que tout est correct
df_cleaned.head()

,store,date,weekly_sales,holiday_flag,temperature,fuel_price,cpi,unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


Les données sont bien chargées et prêtes pour l’analyse.
Je vais maintenant créer les graphiques interactifs pour explorer les tendances.

## Graphiques Interactifs ##

In [5]:
"""

import plotly.graph_objects as go

# 1. Évolution des ventes au fil du temps (tendance globale)
fig_sales_trend = px.line(df_cleaned.groupby("date")["weekly_sales"].sum().reset_index(),
                          x="date", y="weekly_sales",
                          title="Évolution des ventes hebdomadaires",
                          labels={"weekly_sales": "Ventes hebdomadaires", "date": "Date"},
                          template="plotly_dark")

# 2. Comparaison des ventes entre les magasins (moyenne des ventes par magasin)
fig_store_sales = px.bar(df_cleaned.groupby("store")["weekly_sales"].mean().reset_index(),
                         x="store", y="weekly_sales",
                         title="Moyenne des ventes par magasin",
                         labels={"weekly_sales": "Ventes moyennes", "store": "Magasin"},
                         template="plotly_dark")

# 3. Impact des jours fériés sur les ventes
fig_holiday_sales = px.box(df_cleaned, x="holiday_flag", y="weekly_sales",
                           title="Impact des jours fériés sur les ventes",
                           labels={"holiday_flag": "Jour Férié (0 = Non, 1 = Oui)", "weekly_sales": "Ventes hebdomadaires"},
                           template="plotly_dark")

# 4. Relation entre température et ventes
fig_temp_sales = px.scatter(df_cleaned, x="temperature", y="weekly_sales",
                            title="Influence de la température sur les ventes",
                            labels={"temperature": "Température (°F)", "weekly_sales": "Ventes hebdomadaires"},
                            template="plotly_dark", trendline="ols")

# 5. Influence du prix du carburant sur les ventes
fig_fuel_sales = px.scatter(df_cleaned, x="fuel_price", y="weekly_sales",
                            title="Impact du prix du carburant sur les ventes",
                            labels={"fuel_price": "Prix du carburant ($)", "weekly_sales": "Ventes hebdomadaires"},
                            template="plotly_dark", trendline="ols")

# Affichage des graphiques
fig_sales_trend.show()
fig_store_sales.show()
fig_holiday_sales.show()
fig_temp_sales.show()
fig_fuel_sales.show()
"""

'\n\nimport plotly.graph_objects as go\n\n# 1. Évolution des ventes au fil du temps (tendance globale)\nfig_sales_trend = px.line(df_cleaned.groupby("date")["weekly_sales"].sum().reset_index(),\n                          x="date", y="weekly_sales",\n                          title="Évolution des ventes hebdomadaires",\n                          labels={"weekly_sales": "Ventes hebdomadaires", "date": "Date"},\n                          template="plotly_dark")\n\n# 2. Comparaison des ventes entre les magasins (moyenne des ventes par magasin)\nfig_store_sales = px.bar(df_cleaned.groupby("store")["weekly_sales"].mean().reset_index(),\n                         x="store", y="weekly_sales",\n                         title="Moyenne des ventes par magasin",\n                         labels={"weekly_sales": "Ventes moyennes", "store": "Magasin"},\n                         template="plotly_dark")\n\n# 3. Impact des jours fériés sur les ventes\nfig_holiday_sales = px.box(df_cleaned, x="holiday_fla

**Le code ci-dessus a été mis entre triples guillemets pour éviter que les graphiques interactifs ne soient générés automatiquement, ce qui alourdirait le fichier et pourrait dépasser les limites de GitHub.**

J'inclurai les 5 gaphiques au format PNG